In [3]:
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
#import seaborn as sns
import numpy as np
%matplotlib inline

# Lectures



## Information retrieval

* collecting data mostly from user
	* activities - explicit ratings
		- star ratings
		- implementation can vary a lot, calibration (what does star means)
		- more info not especially better here
	* thumbs and like
	* reddit it is only up/down votes
	* pairwise preferences, hybrid, temporary

* when are the ratings provided?
	* consumption
	* memory
	* expectations - for v expensive items, when users don't buy it

* difficulties
	*	accuracy (noise, shifts, change in preferences)
	*	what does ratings mean?
	* more page views than feedbacks
	* use user behaviour to estimate feedback
	* user binary feedback
	* implicit data benefit from large volume and its easier to collect then explicit data
	* purchases and other user actions (implicit consumption-based)
	* follows/friends
	* how we scale and represent actions
	* respect privacy and educate

##Prediction vs recommendation

* prediction - predicts how much u like item
	* helps qualify item
* recommendation - defines items you might like, makes the suggestion
	* we are given items (list of top n) not predictions (%)
	* so it is always tie to interest or activity
	* if you don't like it you don't trust it
	* might feel too pushy - we like our privacy so soft approach might be easier

## computing rank

* how confident are you that item is good?
* what are your risk tolerances?
* specific requirements of community
* using damped means
	- $\frac{\sum_u r_{ui} +k\mu }{ n+k }$
	- without evidence everything is average
* use confidence interval
	- you recommendation is based in lower band of your interval
* use time
	- make earlier votes more important than later votes
	- this will decrease influence of new votes for highly voted items
* damping results so we can influence results for community interest sake 


## Extra reading

* <http://www.evanmiller.org/how-not-to-sort-by-average-rating.html>


# Assignment 1: Non-Personalized Recommenders

Submit a list of the top 5 movies as ranked by a following metrics:

1. Mean Rating: Calculate the mean rating for each movie, order with the highest rating listed first, and submit the top 5.
2. % of ratings 4+: Calculate the percentage of ratings for each movie that are 4 or higher. Order with the highest percentage first, and submit the top 5.
1. Rating Count: Count the number of ratings for each movie, order with the most number of ratings first, and submit the top 5.
1. Top 5 Star Wars: Calculate movies that most often occur with Star Wars: Episode IV - A New Hope (1977) using the $\frac{x+y}{x}$ method described in class. In other words, for each movie, calculate the percentage of Star Wars raters who also rated that movie. Order with the highest percentage first, and submit the top 5.


## Read File

In [48]:
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import copy

#import pdb #debuggera
%matplotlib inline

from __future__ import division #so I can have float as std and int as //

In [60]:
dataFile = "A1Ratings.csv"
print("reading %s" % dataFile)
dfData = pd.read_csv(dataFile, delimiter = ",", error_bad_lines=True, encoding = 'utf-8-sig',na_values = ["NULL",""],header =0);

reading A1Ratings.csv


## Metric one - mean rating
Calculate the mean rating for each movie, order with the highest rating listed first, and submit the top 5.

In [61]:
#dfData.iloc[1] #2nd col (1st movie)
meanRatings = dfData.iloc[:,1:].mean()
meanRatings.sort_values(inplace=True, ascending=False )
print meanRatings.head(5)
#meanRatings.head()

318: Shawshank Redemption, The (1994)             3.600000
260: Star Wars: Episode IV - A New Hope (1977)    3.266667
541: Blade Runner (1982)                          3.222222
1265: Groundhog Day (1993)                        3.166667
593: Silence of the Lambs, The (1991)             3.062500
dtype: float64


## Metric two  - Rating Count: 

Count the number of ratings for each movie, order with the most number of ratings first, and submit the top 5.

In [80]:
ratingCount = dfData.iloc[:,1:].count()
ratingCount.sort_values(inplace=True, ascending=False ) 
ratingCount.head(5)

1: Toy Story (1995)                                        17
593: Silence of the Lambs, The (1991)                      16
260: Star Wars: Episode IV - A New Hope (1977)             15
1210: Star Wars: Episode VI - Return of the Jedi (1983)    14
780: Independence Day (ID4) (1996)                         13
dtype: int64

## Metric three  - % of ratings 4+

Calculate the percentage of ratings for each movie that are 4 or higher and submit the top 5.

In [85]:
positiveRating = dfData.iloc[:,1:].where(dfData>3).count() 
ratingCount = dfData.iloc[:,1:].count()
positiveRating = positiveRating * 100/ratingCount

positiveRating.sort_values(inplace=True, ascending=False)
positiveRating.head(5)

318: Shawshank Redemption, The (1994)             70.000000
260: Star Wars: Episode IV - A New Hope (1977)    53.333333
3578: Gladiator (2000)                            50.000000
541: Blade Runner (1982)                          44.444444
593: Silence of the Lambs, The (1991)             43.750000
dtype: float64

## Metric four  - Top 5 Star Wars

Calculate movies that most often occur with Star Wars: Episode IV - A New Hope (1977) using method described in class. In other words, for each movie, calculate the percentage of Star Wars raters who also rated that movie. Order with the highest percentage first, and submit the top 5.

Method is to count all users that voted both for SW and other movie and divide by no of users that voted for star wars. So its simple Bayean $P(y|SW) =  \frac{P(SW|y)P(y)}{P(SW)}$ = $\frac{count Y\&SW users}{count SW users}$.

How to solve it?

1. Find users that voted for each movie using masks. 
1. Compare masks with each other (using OR as we are only interested in users who seen both (TT)).

In [135]:
#define votes for each movie
maskStarWars = np.logical_not(dfData.iloc[:,1].isnull()) #true means that user voted on movie
maskOthers = np.logical_not(dfData.iloc[:,2:].isnull())

#compare each col with mask
checkSimiliarity = maskOthers.apply(lambda x: x & maskStarWars) 
#need to use OR as only True True counts (person voted on both movies

In [144]:
similiarityToSW = checkSimiliarity.sum()
howManyRatedSW = maskStarWars.sum()

similiarityToSW = similiarityToSW * 100/howManyRatedSW
similiarityToSW[1]
similiarityToSW.sort_values(inplace=True,ascending=False)
similiarityToSW.head(5)

1: Toy Story (1995)                                        93.333333
1210: Star Wars: Episode VI - Return of the Jedi (1983)    86.666667
593: Silence of the Lambs, The (1991)                      80.000000
780: Independence Day (ID4) (1996)                         73.333333
2916: Total Recall (1990)                                  66.666667
dtype: float64